# ETF 투자 전략 비교 분석

이 노트북에서는 3가지 ETF 투자 전략을 백테스트하고 비교합니다:

1. **전략 1: 글로벌 자산배분** - 올웨더 스타일의 분산투자
2. **전략 2: 모멘텀 섹터 로테이션** - 상승 모멘텀 섹터에 집중
3. **전략 3: 배당 + 성장 혼합** - 배당주와 성장주의 균형

## 투자 조건
- 초기 투자금: 420만원
- 월 정기 입금: 30만원
- 리밸런싱: 매월
- 거래 수수료: 0.015%
- 거래세: 0.23% (매도시)

In [ ]:
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Add parent directory to path
sys.path.insert(0, os.path.abspath('..'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Import our modules
from ETFTrading.data.yahoo_loader import YahooETFLoader, generate_sample_data
from ETFTrading.strategy.asset_allocation import GlobalAssetAllocationStrategy
from ETFTrading.strategy.momentum_rotation import MomentumSectorRotationStrategy
from ETFTrading.strategy.dividend_growth import DividendGrowthMixStrategy
from ETFTrading.backtesting.engine import ETFBacktestEngine

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Modules imported successfully!")

## 1. 데이터 수집

Yahoo Finance를 통해 한국 ETF 데이터를 수집합니다.

In [ ]:
# Initialize data loader
loader = YahooETFLoader()

# Define date range
START_DATE = "2020-01-01"
END_DATE = datetime.now().strftime("%Y-%m-%d")

print(f"Fetching data from {START_DATE} to {END_DATE}")
print("-" * 60)

In [ ]:
# Collect all tickers needed for all strategies
strategy1 = GlobalAssetAllocationStrategy()
strategy2 = MomentumSectorRotationStrategy()
strategy3 = DividendGrowthMixStrategy()

all_tickers = set(
    strategy1.get_universe() + 
    strategy2.get_universe() + 
    strategy3.get_universe()
)

print(f"Total ETFs to fetch: {len(all_tickers)}")
print(f"Tickers: {sorted(all_tickers)}")

In [ ]:
# Load data
try:
    data = loader.load_multiple(
        tickers=list(all_tickers),
        start_date=START_DATE,
        end_date=END_DATE,
        use_cache=True
    )
    print(f"\nSuccessfully loaded data for {len(data)} ETFs")
    
    # Show sample
    if data:
        sample_ticker = list(data.keys())[0]
        print(f"\nSample data for {sample_ticker}:")
        print(data[sample_ticker].head())
        print(f"\nDate range: {data[sample_ticker]['date'].min()} to {data[sample_ticker]['date'].max()}")
        
except Exception as e:
    print(f"Error loading data: {e}")
    print("\nGenerating sample data for demonstration...")
    data = generate_sample_data(
        tickers=list(all_tickers),
        start_date=START_DATE,
        end_date=END_DATE
    )
    print(f"Generated sample data for {len(data)} ETFs")

## 2. 전략 설명

In [ ]:
print("=" * 80)
print("STRATEGY 1: Global Asset Allocation")
print("=" * 80)
print(strategy1.describe())
print("\n" * 2)

print("=" * 80)
print("STRATEGY 2: Momentum Sector Rotation")
print("=" * 80)
print(strategy2.describe())
print("\n" * 2)

print("=" * 80)
print("STRATEGY 3: Dividend + Growth Mix")
print("=" * 80)
print(strategy3.describe())

## 3. 백테스팅 실행

In [ ]:
# Initialize backtesting engine
engine = ETFBacktestEngine(
    initial_capital=4_200_000,
    monthly_deposit=300_000,
    commission_rate=0.00015,
    tax_rate=0.0023,
    rebalance_frequency="monthly"
)

print("Backtesting engine initialized")
print(f"Initial capital: {engine.initial_capital:,.0f} KRW")
print(f"Monthly deposit: {engine.monthly_deposit:,.0f} KRW")
print(f"Commission rate: {engine.commission_rate:.4%}")
print(f"Tax rate: {engine.tax_rate:.3%}")

In [ ]:
# Run backtest for Strategy 1
print("\nRunning backtest for Strategy 1: Global Asset Allocation...")
equity1 = engine.run(strategy1, data, START_DATE, END_DATE)
stats1 = engine.get_summary_stats(equity1)
trades1 = engine.get_trade_log()
print(f"Completed! {len(equity1)} days, {len(trades1)} trades")

In [ ]:
# Run backtest for Strategy 2
print("\nRunning backtest for Strategy 2: Momentum Sector Rotation...")
equity2 = engine.run(strategy2, data, START_DATE, END_DATE)
stats2 = engine.get_summary_stats(equity2)
trades2 = engine.get_trade_log()
print(f"Completed! {len(equity2)} days, {len(trades2)} trades")

In [ ]:
# Run backtest for Strategy 3
print("\nRunning backtest for Strategy 3: Dividend + Growth Mix...")
equity3 = engine.run(strategy3, data, START_DATE, END_DATE)
stats3 = engine.get_summary_stats(equity3)
trades3 = engine.get_trade_log()
print(f"Completed! {len(equity3)} days, {len(trades3)} trades")

## 4. 성과 비교

In [ ]:
# Create comparison table
comparison = pd.DataFrame({
    'Strategy 1\n(Asset Allocation)': [
        f"{stats1['initial_value']:,.0f}",
        f"{stats1['final_value']:,.0f}",
        f"{stats1['total_return']:.2%}",
        f"{stats1['cagr']:.2%}",
        f"{stats1['volatility']:.2%}",
        f"{stats1['sharpe_ratio']:.2f}",
        f"{stats1['max_drawdown']:.2%}",
        f"{stats1['win_rate']:.2%}",
        f"{len(trades1)}"
    ],
    'Strategy 2\n(Momentum Rotation)': [
        f"{stats2['initial_value']:,.0f}",
        f"{stats2['final_value']:,.0f}",
        f"{stats2['total_return']:.2%}",
        f"{stats2['cagr']:.2%}",
        f"{stats2['volatility']:.2%}",
        f"{stats2['sharpe_ratio']:.2f}",
        f"{stats2['max_drawdown']:.2%}",
        f"{stats2['win_rate']:.2%}",
        f"{len(trades2)}"
    ],
    'Strategy 3\n(Dividend+Growth)': [
        f"{stats3['initial_value']:,.0f}",
        f"{stats3['final_value']:,.0f}",
        f"{stats3['total_return']:.2%}",
        f"{stats3['cagr']:.2%}",
        f"{stats3['volatility']:.2%}",
        f"{stats3['sharpe_ratio']:.2f}",
        f"{stats3['max_drawdown']:.2%}",
        f"{stats3['win_rate']:.2%}",
        f"{len(trades3)}"
    ]
}, index=[
    'Initial Value',
    'Final Value',
    'Total Return',
    'CAGR',
    'Volatility',
    'Sharpe Ratio',
    'Max Drawdown',
    'Win Rate',
    'Number of Trades'
])

print("\n" + "=" * 80)
print("PERFORMANCE COMPARISON")
print("=" * 80)
print(comparison.to_string())
print("=" * 80)

## 5. 시각화

In [ ]:
# Plot equity curves
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# 1. Equity curves
ax = axes[0, 0]
ax.plot(equity1['date'], equity1['portfolio_value'] / 1_000_000, 
        label='Strategy 1: Asset Allocation', linewidth=2)
ax.plot(equity2['date'], equity2['portfolio_value'] / 1_000_000, 
        label='Strategy 2: Momentum Rotation', linewidth=2)
ax.plot(equity3['date'], equity3['portfolio_value'] / 1_000_000, 
        label='Strategy 3: Dividend+Growth', linewidth=2)
ax.set_title('Portfolio Value Over Time', fontsize=14, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Portfolio Value (Million KRW)')
ax.legend()
ax.grid(True, alpha=0.3)

# 2. Drawdown
ax = axes[0, 1]
for equity, label in [(equity1, 'Strategy 1'), (equity2, 'Strategy 2'), (equity3, 'Strategy 3')]:
    cummax = equity['portfolio_value'].cummax()
    drawdown = (equity['portfolio_value'] - cummax) / cummax
    ax.plot(equity['date'], drawdown * 100, label=label, linewidth=2)
ax.set_title('Drawdown (%)', fontsize=14, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Drawdown (%)')
ax.legend()
ax.grid(True, alpha=0.3)

# 3. Rolling returns
ax = axes[1, 0]
window = 60  # 60-day rolling
for equity, label in [(equity1, 'Strategy 1'), (equity2, 'Strategy 2'), (equity3, 'Strategy 3')]:
    returns = equity['portfolio_value'].pct_change()
    rolling_ret = returns.rolling(window).mean() * window * 100
    ax.plot(equity['date'], rolling_ret, label=label, linewidth=2)
ax.set_title(f'{window}-Day Rolling Returns (%)', fontsize=14, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Return (%)')
ax.legend()
ax.grid(True, alpha=0.3)

# 4. Performance metrics bar chart
ax = axes[1, 1]
metrics = ['CAGR', 'Sharpe', 'Max DD']
strat1_metrics = [stats1['cagr'] * 100, stats1['sharpe_ratio'], stats1['max_drawdown'] * 100]
strat2_metrics = [stats2['cagr'] * 100, stats2['sharpe_ratio'], stats2['max_drawdown'] * 100]
strat3_metrics = [stats3['cagr'] * 100, stats3['sharpe_ratio'], stats3['max_drawdown'] * 100]

x = np.arange(len(metrics))
width = 0.25

ax.bar(x - width, strat1_metrics, width, label='Strategy 1')
ax.bar(x, strat2_metrics, width, label='Strategy 2')
ax.bar(x + width, strat3_metrics, width, label='Strategy 3')

ax.set_title('Performance Metrics Comparison', fontsize=14, fontweight='bold')
ax.set_ylabel('Value')
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 6. 거래 분석

In [ ]:
# Analyze trades for each strategy
for i, (trades, name) in enumerate([
    (trades1, "Strategy 1: Asset Allocation"),
    (trades2, "Strategy 2: Momentum Rotation"),
    (trades3, "Strategy 3: Dividend+Growth")
], 1):
    print(f"\n{'='*60}")
    print(f"{name}")
    print(f"{'='*60}")
    
    if trades.empty:
        print("No trades executed")
        continue
    
    print(f"Total trades: {len(trades)}")
    print(f"Buy orders: {(trades['side'] == 'buy').sum()}")
    print(f"Sell orders: {(trades['side'] == 'sell').sum()}")
    print(f"Total commission: {trades['commission'].sum():,.0f} KRW")
    print(f"Total tax: {trades['tax'].sum():,.0f} KRW")
    
    print("\nMost traded ETFs:")
    print(trades['ticker'].value_counts().head(5))
    
    print("\nSample trades:")
    print(trades.head(10).to_string(index=False))

## 7. 결론 및 추천

### 전략별 특징:

**Strategy 1: Global Asset Allocation**
- 장점: 안정적, 낮은 변동성, 자동 리밸런싱
- 단점: 상대적으로 낮은 수익률
- 추천 대상: 안정적 투자를 원하는 보수적 투자자

**Strategy 2: Momentum Sector Rotation**
- 장점: 높은 수익 잠재력, 트렌드 추종
- 단점: 높은 변동성, 잦은 거래, 높은 비용
- 추천 대상: 적극적 투자를 원하는 공격적 투자자

**Strategy 3: Dividend + Growth Mix**
- 장점: 배당 수익 + 성장성, 중간 리스크
- 단점: 미국 시장 집중, 환율 리스크
- 추천 대상: 균형잡힌 투자를 원하는 중도 투자자

### 다음 단계:
1. 실제 모의투자 계좌로 테스트
2. 월 단위 자동 리밸런싱 설정
3. 정기적인 성과 모니터링
4. 시장 상황에 따른 전략 조정